In [33]:
# En este archivo definimos las funciones que empleará nuestro sistema de web_scrapping

# Declaramos las librerías que usaremos
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException


# Nuestro dataset estará formado por una lista (documento) de listas (filas), que posteriormente
# transformaremos a formato CSV. Rellenamos la primera fila con los nombres de las cabeceras
# (los campos "*_upper" y "*_lower" se refieren a los márgenes de error superior e inferior, respectivamente)
dataset = [['name', 'version', 'release', 'gps', 'mass_1', 'mass_1_upper', 'mass_1_lower', 'mass_2', 'mass_2_upper',
            'mass_2_lower', 'network_snr', 'network_snr_upper', 'network_snr_lower', 'distance', 'distance_upper',
            'distance_lower', 'chi_eff', 'chi_eff_upper', 'chi_eff_lower', 'total_mass', 'total_mass_upper', 
            'total_mass_lower', 'chirp_mass', 'chirp_mass_upper', 'chirp_mass_lower', 'detector_frame_chirp_mass', 
            'detector_frame_chirp_mass_upper', 'detector_frame_chirp_mass_lower', 'redshift', 'redshift_upper', 
            'redshift_lower', 'false_alarm_rate', 'p_astro', 'final_mass', 'final_mass_upper', 'final_mass_lower']]


def execute_form(fecha_inicio, fecha_fin):
    """Función que rellena y ejecuta el formulario"""
    # Declaramos nuestro navegador
    driver = webdriver.Chrome()

    # Abrimos nuestra web
    driver.get('https://www.gw-openscience.org')

    # Seleccionamos en el menú eventos y catálogos
    elem = driver.find_element(By.XPATH, "//a[contains(text(), 'Events and Catalogs')]")   

    # Hacemos Click
    driver.execute_script("arguments[0].click();", elem)
    # Esperamos a que cargue
    driver.implicitly_wait(10)

    # Pulsamos el botón query de la ventana que acaba de cargar
    elem2 = driver.find_element(By.XPATH, "//button[text()='Query']").click() 
    # Esperamos a que se cargue el formulario
    driver.implicitly_wait(10)

    # Vamos a rellenar el formulario
    # Comenzamos por los selectores
    select = Select(driver.find_element(By.ID,'mul-release-sel'))

    # Vamos a seleccionar elementos por su nombre, son las diferentes releases
    select.select_by_visible_text('GWTC-1-marginal')
    select.select_by_visible_text('GWTC-1-confident')
    select.select_by_visible_text('O3_IMBH_marginal')
    select.select_by_visible_text('GWTC-2')
    select.select_by_visible_text('GWTC-2.1-marginal')
    select.select_by_visible_text('GWTC-2.1-confident')
    select.select_by_visible_text('GWTC-3-marginal')
    select.select_by_visible_text('GWTC-3-confident')
    select.select_by_visible_text('Initial_LIGO_Virgo')
    
    # Ahora metemos las fechas "desde" y "hasta"
    imputFecha1 = driver.find_element(By.XPATH,"//div[@id='min-datetime-tbox']/input")
    imputFecha1.send_keys(fecha_inicio+'T00:00:00')  
    imputFecha2 = driver.find_element(By.XPATH,"//div[@id='max-datetime-tbox']/input")
    imputFecha2.send_keys(fecha_fin+'T00:00:00')  
    
    # Ahora marcamos el checkbox para que nos den la última versión de las detecciones
    # De este modo no tendremos repetidos
    checkbox = driver.find_element(By.XPATH,"//label[@id='version-tbox']/input")
    # Hacemos Click para marcar el checkbox
    driver.execute_script("arguments[0].click();", checkbox)
    
    # Ya tenemos todo, pulsamos el botón submit para generar la tabla con los datos
    # Localizamos el botón
    boton = driver.find_element(By.ID, 'submit-query-btn')
    # Lo pulsamos
    driver.execute_script("arguments[0].click();", boton)
    # Esperamos a que se cargue la tabla
    driver.implicitly_wait(10)
    
    # Cuidado que no selecciona todas las columnas, debemos seleccionar lo que deseamos ver
    # Pulsamos el botón para desplegar el selector de columnas
    desplegable = driver.find_element(By.CLASS_NAME ,'tablesaw-columntoggle-btnwrap').click()
    
    # Ahora ya vemos los selectores de columnas, comprobaremos que estén todos seleccionados,
    # pulsando aquellos que no lo estén.
    # Creamos una función para evitar la duplicación de código en esta parte
    def click_checkboxes(*checkboxes_texts):
        for text in checkboxes_texts:
            # Localizamos el checkbox
            checkbox = driver.find_element(By.XPATH, f"//label[text()='{text}']/input")
            # Comprobamos si está checkeado
            if checkbox.get_attribute("checked") != "true":
                # Hacemos Click para marcar el checkbox
                driver.execute_script("arguments[0].click();", checkbox)
    
    # Utilizamos dicha función para marcar todas las casillas (columnas) deseadas
    click_checkboxes('Version', 'Release', 'GPS', 'Mass 1 (M☉)', 'Mass 2 (M☉)', 'Network SNR',
                     'Distance (Mpc)', 'χeff', 'Total Mass (M☉)', 'Chirp Mass (M☉)', 
                     'Detector Frame Chirp Mass (M☉)', 'Redshift', 'False Alarm Rate (yr-1)', 
                     'Pastro', 'Final Mass (M☉)')
        
    # Pulsamos el botón para ocultar el desplegable y poder trabajar cómodamente
    desplegable = driver.find_element(By.CLASS_NAME ,'tablesaw-columntoggle-btnwrap').click()
    
    # Ya tenemos la tabla en pantalla, ahora la recorreremos para obtener los datos
    # Primeramente, obtenemos cada una de sus filas (excluyendo la primera, las cabeceras)
    detecciones = driver.find_elements(By.CSS_SELECTOR, "tr")[1:]

    # Por cada detección (fila) trataremos cada una de sus columnas. Algunas de estas columnas
    # merecen un trato especial, como es el caso de las columnas numéricas que pueden tener
    # valores de error superior e inferior. Creamos pues una función para tratarlas:

    def get_value_upper_lower(field):
        # Tomamos el valor del campo, desechando el valor numérico "escondido" usado para ordenar,
        # y los demás posibles valores restantes (errores superior e inferior)
        value = field.text.split(' ')[1].split('\n')[0]
        # Obtenemos el valor del error superior (si existe)
        try:
            upper = field.find_element(By.CSS_SELECTOR, "sup").text
        except NoSuchElementException:
            upper = None
        # Obtenemos el valor del error inferior (si existe)
        try:
            lower = field.find_element(By.CSS_SELECTOR, "sub").text
        except NoSuchElementException:
            lower = None
        # Retornamos los 3 valores
        return value, upper, lower

    # Hay otro tipo de campos que pueden contener valores con espacios (ej.: "≥ 0.99"). También
    # cuidaremos de tratar estos mediante esta función:

    def get_value_potential_spaces(field):
        # Tomamos el valor del campo, desechando el valor numérico "escondido" usado para ordenar,
        # y tomando todo aquello que siga detrás
        return ' '.join(field.text.split(' ')[1:])


execute_form('20150101','20221105')


('0.62', None, None)